In [1]:
import warnings; warnings.filterwarnings('ignore')

In [2]:
from growing_instability_lib import *

Using TensorFlow backend.


In [3]:
sample_sub = pd.read_csv('../data/sampleSubmission.csv')
topics = sorted(set(sample_sub.columns.difference(['id'])))

topic2actual = {}
for i in sample_sub.columns:
    if 'id' == i:
        continue
    topic2actual[i] = segment(i)
    
target_columns = sorted(topics)
len(topics)

160

In [4]:
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.lsimodel import LsiModel
from gensim.corpora import Dictionary

dictionary = TfidfModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.dictionary')
tfidf = TfidfModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.tfidf')
lsi = LsiModel.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.lsi')

In [5]:
wvmodel = Word2Vec.load('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.w2v.model')

In [6]:
fsmodel = fasttext.load_model('corpus/train_body_data-with_labels_False-retain_special_chars_False.with_test_data.csv.fasttext.model.bin')

In [7]:
topic2actual

{'activism': ['activism'],
 'afghanistan': ['afghanistan'],
 'aid': ['aid'],
 'algerianhostagecrisis': ['algerian', 'hostage', 'crisis'],
 'alqaida': ['al', 'qaida'],
 'alshabaab': ['al', 'shaba', 'ab'],
 'antiwar': ['antiwar'],
 'arabandmiddleeastprotests': ['arab', 'and', 'middle', 'east', 'protests'],
 'armstrade': ['arms', 'trade'],
 'australianguncontrol': ['australian', 'gun', 'control'],
 'australiansecurityandcounterterrorism': ['australian',
  'security',
  'and',
  'counterterrorism'],
 'bastilledaytruckattack': ['bastille', 'day', 'truck', 'attack'],
 'belgium': ['belgium'],
 'berlinchristmasmarketattack': ['berlin', 'christmas', 'market', 'attack'],
 'bigdata': ['big', 'data'],
 'biometrics': ['biometrics'],
 'bokoharam': ['boko', 'haram'],
 'bostonmarathonbombing': ['boston', 'marathon', 'bombing'],
 'britisharmy': ['british', 'army'],
 'brusselsattacks': ['brussels', 'attacks'],
 'cameroon': ['cameroon'],
 'carers': ['carers'],
 'charliehebdoattack': ['charlie', 'hebdo', 

In [8]:
# manual_topics = {
#     'alshabaab': ['al', 'shabaab'],
# #     'earthquakes': ['earthquake'],
# #     'firefighters': ['firefighter'],
#     'rio20earthsummit': ['rio', '20', 'earth', 'summit'],
#     'zikavirus': ['zika']
# }

# topic2actual.update(manual_topics)

In [9]:
wvmodel.most_similar('shabaab')

[(u'qaida', 0.7521548271179199),
 (u'shabab', 0.750149667263031),
 (u'aqim', 0.7214016914367676),
 (u'islamist', 0.7179936170578003),
 (u'boko', 0.7109889388084412),
 (u'qaeda', 0.7079288363456726),
 (u'nusra', 0.6980888843536377),
 (u'taliban', 0.6857478022575378),
 (u'somalia', 0.6842936873435974),
 (u'militants', 0.6828135251998901)]

# Perform exploratory unsupervised discovery of unseen topics

In [10]:
%%time
# wvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'wvec_trainingX')
# fvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'fvec_trainingX')

# tfidf_wvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_wvec_trainingX')
# tfidf_fvec_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_fvec_trainingX')
# tfidf_lsi_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'tfidf_lsi_trainingX')

# word2idx_trainingX = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'word2idx_trainingX')
_word2idx = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', '_word2idx')
trainingY = pd.read_hdf('training_data_wv_fs_lsi_no_stopwords.hdf', 'trainingY')

CPU times: user 1.46 s, sys: 712 ms, total: 2.17 s
Wall time: 2.6 s


In [11]:
training_topics_freq = trainingY.sum()
training_topics_freq.head()

activism                     NaN
afghanistan              10032.0
aid                       1891.0
algerianhostagecrisis       58.0
alqaida                   3194.0
dtype: float64

In [12]:
unseen_topics = set(topics).intersection(training_topics_freq.ix[training_topics_freq.isnull()].index)

In [13]:
unseen_topics

{'activism',
 'bastilledaytruckattack',
 'berlinchristmasmarketattack',
 'brusselsattacks',
 'charliehebdoattack',
 'francetrainattack',
 'munichshooting',
 'orlandoterrorattack',
 'parisattacks',
 'peaceandreconciliation',
 'sanbernardinoshooting',
 'tunisiaattack2015',
 'turkeycoupattempt',
 'zikavirus'}

In [14]:
ind2word = {j:i for i, j in _word2idx.iteritems()}
ind2class = dict(enumerate(topics))
class2ind = {j: i for i, j in ind2class.items()}

## Use idf only in tfidf weighted vectors

In [17]:
_fsmodel_cache = {}
def get_fsvec(word):
    if word in _fsmodel_cache:
        fv = _fsmodel_cache[word]
    else:
        fv = fsmodel[word]
        _fsmodel_cache[word] = fv

    return fv

In [18]:
def transform_tfidf_word2vec(tokens, stopwords=[], trim=True):
#     global wvmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    )
    
    if trim:
        wv_feature_vec = wv_feature_vec.map(
        lambda x: filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
        )
    else:
        wv_feature_vec = wv_feature_vec.map(
        lambda x: tfidf[dictionary.doc2bow(x)]
        )
        
    wv_feature_vec = wv_feature_vec.map(
        lambda x: np.array([wvmodel[dictionary.id2token[id]] * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return wv_feature_vec


def transform_tfidf_fasttext(tokens, stopwords=[], trim=True):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    )
    
    if trim:
        fs_feature_vec = fs_feature_vec.map(
        lambda x: filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
        )
    else:
        fs_feature_vec = fs_feature_vec.map(
        lambda x: tfidf[dictionary.doc2bow(x)]
        )
        
    fs_feature_vec = fs_feature_vec.map(
        lambda x: np.array([np.array(get_fsvec(dictionary.id2token[id])) * w for id, w in x]).mean(axis=0) if len(x) > 0 else np.nan
    )

    return fs_feature_vec


def build_lsi_vector(x, trim=True):
    v = np.zeros(lsi.num_topics)
    if trim:
        corpus = filter(lambda s: s[1] > 0.1, sorted(tfidf[dictionary.doc2bow(x)], key=lambda y: y[1], reverse=True))
    else:
        corpus = tfidf[dictionary.doc2bow(x)]

    for ix, vv in lsi[corpus]:
        v[ix] = vv

    return v


def transform_tfidf_lsi(tokens, stopwords=[], trim=True):
#     global fsmodel
#     global tfidf
    # This requires wvmodel to be present in the namespace.
    lsi_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(
        lambda x: build_lsi_vector(x, trim) if len(x) > 0 else np.nan
    )

    return lsi_feature_vec


def transform_fasttext(tokens, stopwords=[]):
    global fsmodel
    # This requires fsmodel to be present in the namespace.
    fs_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords)]
    ).map(lambda x: np.array([get_fsvec(w) for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return fs_feature_vec


def transform_word2vec(tokens, stopwords=[]):
    global wvmodel
    # This requires wvmodel to be present in the namespace.
    wv_feature_vec = tokens.map(
        lambda x: [w for w in x.split() if (w not in stopwords and w in wvmodel.wv.vocab)]
    ).map(lambda x: np.array([wvmodel[w] for w in x]).mean(axis=0) if len(x) > 0 else np.nan)

    return wv_feature_vec


def parallel_generate_word_vectors(samp, transformer, stopwords, batch, num_proc):
    with Parallel(n_jobs=num_proc) as parallel:
        dataset = []
        is_break = False
        i = 0

        while not is_break:
            payload = []

            for j in xrange(num_proc):
                t_df = samp[(i + j) * batch: (i + 1 + j) * batch]

                if t_df.empty:
                    is_break = True
                    continue

                payload.append(
                    delayed(transformer)(
                        t_df, stopwords
                    )
                )

            print('Current batch in main thread: {}'.format((i + j) * batch))

            if payload:
                results = parallel(payload)
                dataset.extend(results)
                i += num_proc

    return pd.concat(dataset)

In [19]:
def word2idx_transform(word, _word2idx):
    return _word2idx.get(word, 0)

def extract_features_for(df, min_batch=2000, stopwords=[], num_proc=7):
    df_tokens = transform_text(df)
    
    batch = min(df_tokens.shape[0] / num_proc, min_batch)

    print('Computing fs features...')
    fvec = parallel_generate_word_vectors(df_tokens, transform_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing wv features...')
    wvec = parallel_generate_word_vectors(df_tokens, transform_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Mapping word indices...')
#     word_indices = []
    word_indices = df_tokens.map(lambda x: [word2idx_transform(i, _word2idx) for i in x.split()])

    print('Computing tfidf fs features...')
    tfidf_fvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_fasttext, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing tfidf wv features...')
    tfidf_wvec = parallel_generate_word_vectors(df_tokens, transform_tfidf_word2vec, stopwords=stopwords, batch=batch, num_proc=num_proc)

    print('Computing tfidf lsi features...')
    tfidf_lsi = parallel_generate_word_vectors(df_tokens, transform_tfidf_lsi, stopwords=stopwords, batch=batch, num_proc=num_proc)
    
    return word_indices, wvec, fvec, tfidf_wvec, tfidf_fvec, tfidf_lsi

In [20]:
with open('../data/TestData.json') as fl:
    data = json.load(fl)
    test_df = pd.DataFrame(data['TestData']).T
    del(data)

In [ ]:
'munichshooting': [array([6268, 6254, 6231, 6267, 6232]),
  array([4031, 6268, 6231, 2129, 6254]),
  array([6239, 6254, 6249, 6231, 6253])],

In [21]:
test_transformed_text = transform_text(test_df)

In [22]:
%%time
test_word_indices,test_wvec, test_fvec, test_tfidf_wvec, test_tfidf_fvec, test_tfidf_lsi = extract_features_for(
    test_df
)

Computing fs features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing wv features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Mapping word indices...
Computing tfidf fs features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing tfidf wv features...
Current batch in main thread: 6498
Current batch in main thread: 14079
Computing tfidf lsi features...
Current batch in main thread: 6498
Current batch in main thread: 14079
CPU times: user 40.9 s, sys: 3.24 s, total: 44.2 s
Wall time: 1min 6s


In [23]:
def filter_stopwords(x, stopwords=['and', 'day', 'of', 'in']):
    return [i for i in x if i not in stopwords]

tpa = pd.DataFrame(topic2actual.items(), columns=['topics', 'actual'])

tpa['wv'] = transform_word2vec(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))))
tpa['fs'] = transform_fasttext(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))))

tpa['tk'] = tpa.actual.map(lambda x: [word2idx_transform(i, _word2idx) for i in x])

tpa['tfidf_wv'] = transform_tfidf_word2vec(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)
tpa['tfidf_fs'] = transform_tfidf_fasttext(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)
tpa['tfidf_lsi'] = transform_tfidf_lsi(tpa.actual.map(lambda x: ' '.join(filter_stopwords(x))), trim=False)

unseen_topics2vec = tpa[tpa.topics.isin(unseen_topics)]
unseen_topics2vec = unseen_topics2vec.set_index('topics')
unseen_topics2vec = unseen_topics2vec.ix[sorted(unseen_topics)]

all_topics2vec = tpa[tpa.topics.isin(topics)]
all_topics2vec = all_topics2vec.set_index('topics')
all_topics2vec = all_topics2vec.ix[sorted(topics)]

In [24]:
null_index = test_wvec[test_wvec.isnull()].index
if any(null_index):
    test_wvec.ix[null_index] = [[0] * wvmodel.vector_size] * 3
print(len(null_index))
    
null_index = test_fvec[test_fvec.isnull()].index
if any(null_index):
    test_fvec.ix[null_index] = [[0] * fsmodel.dim] * 3
print(len(null_index))

null_index = test_tfidf_wvec[test_tfidf_wvec.isnull()].index
if any(null_index):
    test_tfidf_wvec.ix[null_index] = [[0] * wvmodel.vector_size] * 3
print(len(null_index))
    
null_index = test_tfidf_fvec[test_tfidf_fvec.isnull()].index
if any(null_index):
    test_tfidf_fvec.ix[null_index] = [[0] * fsmodel.dim] * 3
print(len(null_index))
    
null_index = test_tfidf_lsi[test_tfidf_lsi.isnull()].index
if any(null_index):
    test_tfidf_lsi.ix[null_index] = [[0] * lsi.num_topics] * 3
print(len(null_index))


3
3
3
3
3


In [25]:
unsupervised_unseen_topic_detection = {
    'tfidf_wvec': [test_tfidf_wvec, unseen_topics2vec.tfidf_wv],
    'tfidf_fvec': [test_tfidf_fvec, unseen_topics2vec.tfidf_fs],
    'tfidf_lsi': [test_tfidf_lsi, unseen_topics2vec.tfidf_lsi],
#     'tk': 
}

unsupervised_all_topic_detection = {
    'tfidf_wvec': [test_tfidf_wvec, all_topics2vec.tfidf_wv],
    'tfidf_fvec': [test_tfidf_fvec, all_topics2vec.tfidf_fs],
    'tfidf_lsi': [test_tfidf_lsi, all_topics2vec.tfidf_lsi],
#     'tk': 
}

In [26]:
# N = 10
# unseen_topics_rank = {}

# for key, (tstvec, tpvec) in unsupervised_unseen_topic_detection.items():
#     sim = cosine_similarity(
#         np.vstack(tstvec),
#         np.vstack(tpvec),
#     )

#     for i, t in enumerate(unseen_topics2vec.index):
#         idx = sim[:, i].argsort()[::-1][:N]
#         v = sim[:, i][idx]

#         unseen_topics_rank.setdefault(t, []).append(idx)

In [27]:
N = 10
topics_rank = {}
test_max_class_df = pd.DataFrame()
test_max_score_df = pd.DataFrame()
test_max_class_score_df = pd.DataFrame()

for key, (tstvec, tpvec) in unsupervised_unseen_topic_detection.items():
    sim = cosine_similarity(
        np.vstack(tstvec),
        np.vstack(tpvec),
    )

#     for i, t in enumerate(all_topics2vec.index):
#         idx = sim[:, i].argsort()[::-1][:N]
#         v = sim[:, i][idx]

#         topics_rank.setdefault(t, []).append(idx)
    test_max_class_score_df[key] = pd.Series(zip(tpvec.index[sim.argmax(axis=1)], sim.max(axis=1)), index=tstvec.index)
    test_max_score_df[key] = pd.Series(sim.max(axis=1), index=tstvec.index)
    test_max_class_df[key] = pd.Series(tpvec.index[sim.argmax(axis=1)], index=tstvec.index)


In [28]:
# cosine_similarity(test_tfidf_wvec.iloc[4489], np.vstack(all_topics2vec.tfidf_wv))

In [29]:
sc = StandardScaler()

In [30]:
# test_max_score_df_normed = pd.DataFrame(sc.fit_transform(test_max_score_df), columns=test_max_score_df.columns, index=test_max_score_df.index)
test_max_score_df_normed = test_max_score_df

In [31]:
from collections import Counter

In [305]:
g = test_max_score_df_normed.max(axis=1)
g = g[g > 0.65]
w = test_max_class_df.ix[g.index]

In [306]:
sh = pd.DataFrame()
q = w[w.apply(lambda x: min(x) == max(x), 1)]
# q = w[w.apply(lambda x: len(set(x)) <= 2, 1)]
sh['class'] = q.apply(max, 1)
# sh['class'] = q.apply(lambda x: Counter(x).most_common()[0][0], 1)
sh['scores'] = g

In [307]:
sh = sh.sort_values(['class', 'scores'])
sh['order'] = sh.index.str.replace('TestData_', '').map(int)

In [308]:
# for topic, group in sh.groupby('class'):
#     group = group.sort_values(['scores'], ascending=False)[:20]
#     print topic, group['order'].median(), group['order'].quantile(0.25), group['order'].quantile(0.75), group.shape[0]

In [309]:
def keyword_match(group, topic, min_weight=4.85):
    def get_top_token_count(x):
        q = [
            (tfidf.idfs[dictionary.token2id[i]], i) for i in set(
                tpa[tpa.topics == topic].actual.values[0]
            ) if tfidf.idfs[dictionary.token2id[i]] > min_weight
        ]
        
        c = 0
        if q:
            c = x.count(sorted(q)[-1][1])
        
        return c
    
    return test_transformed_text.ix[group.index].str.split().map(
#         lambda x: sum(
#             [tfidf.idfs[dictionary.token2id[i]] for i in set(
#                 tpa[tpa.topics == topic].actual.values[0]
#             ).intersection(x)]
#         )
        get_top_token_count
    )

In [310]:
l = [3, 2, 6, 4, 7, 3]
l.count(0)

0

In [311]:
for t in unseen_topics:
    print sorted([(tfidf.idfs[dictionary.token2id[i]], i) for i in set(tpa[tpa.topics == t].actual.values[0])])[-1]

(4.835135466946886, 'train')
(7.835391568174351, 'reconciliation')
(6.900672593783962, 'munich')
(11.662759885031802, 'hebdo')
(6.1978518843233195, 'berlin')
(12.589050198848792, 'bernardino')
(8.329532383735154, 'activism')
(6.742743260587802, 'coup')
(10.63824573631232, 'bastille')
(4.86092117192136, 'paris')
(8.664578943450694, 'tunisia')
(15.662759885031802, 'zika')
(6.307693021487954, 'brussels')
(8.809601273361073, 'orlando')


In [312]:
keyword_match(group, topic)

TestData_04490    18
TestData_07478     3
TestData_04550     4
TestData_06610     5
TestData_07263     2
Name: bodyText, dtype: int64

In [345]:
N = 100
min_N = 5
max_N = 15
train_test_data = {}
inferred_topic_indices = []
for topic, group in sh.groupby('class'):
    group = group.copy()
    group = group.sort_values(['scores'], ascending=False)[:N]

    if group.shape[0] < min_N:
        # Very small sample won't have a bog effect anyway.
        train_test_data[topic] = sorted(group.index)
        print topic, group.shape[0]
    else:

        keyword_weight = keyword_match(group, topic)
        group['kw'] = keyword_weight
        group = group[group.kw > 0]        

        minb = group['order'].quantile(0.1)
        maxb = group['order'].quantile(0.9)

        print topic, minb, maxb
        if group.shape[0] < min_N:
            train_test_data[topic] = sorted(group.index)
            print topic, group.shape[0]
        else:
            group = group[group.order.between(minb, maxb)]
            
        #     train_test_data[topic] = sorted(group.index)
            train_test_data[topic] = sorted(group.sort_values('kw', ascending=False)[:max_N].index)

    inferred_topic_indices.extend(train_test_data[topic])

train_test_df = test_df.ix[inferred_topic_indices].copy()


bastilledaytruckattack 6106.8 6619.0
berlinchristmasmarketattack 5966.8 7531.2
brusselsattacks 2207.6 5822.4
charliehebdoattack 66.5 4203.4
francetrainattack nan nan
francetrainattack 0
munichshooting 5614.9 6268.3
orlandoterrorattack 5736.0 5871.2
parisattacks 2449.0 6132.5
peaceandreconciliation 4
sanbernardinoshooting 3790.5 5079.0
tunisiaattack2015 819.0 4821.0
turkeycoupattempt 5442.0 7143.0
zikavirus 4508.0 7217.6
zikavirus 4


In [346]:
group

,class,scores,order,kw
TestData_04490,zikavirus,0.997918,4490,18
TestData_07478,zikavirus,0.900109,7478,3
TestData_04550,zikavirus,0.873176,4550,4
TestData_06610,zikavirus,0.681341,6610,5


In [347]:
sh[sh['class'] == 'turkeycoupattempt']

,class,scores,order
TestData_02767,turkeycoupattempt,0.651322,2767
TestData_04933,turkeycoupattempt,0.652083,4933
TestData_04648,turkeycoupattempt,0.654511,4648
TestData_04363,turkeycoupattempt,0.656561,4363
TestData_01986,turkeycoupattempt,0.661199,1986
TestData_01932,turkeycoupattempt,0.661573,1932
TestData_06149,turkeycoupattempt,0.661752,6149
TestData_05076,turkeycoupattempt,0.663739,5076
TestData_04448,turkeycoupattempt,0.663858,4448
TestData_02202,turkeycoupattempt,0.665919,2202


In [348]:
train_test_data

{'bastilledaytruckattack': [u'TestData_06107',
  u'TestData_06122',
  u'TestData_06126',
  u'TestData_06155',
  u'TestData_06158',
  u'TestData_06171',
  u'TestData_06397'],
 'berlinchristmasmarketattack': [u'TestData_07490',
  u'TestData_07504',
  u'TestData_07514',
  u'TestData_07515',
  u'TestData_07522'],
 'brusselsattacks': [u'TestData_03280',
  u'TestData_03349',
  u'TestData_03498',
  u'TestData_03514',
  u'TestData_03555',
  u'TestData_03564',
  u'TestData_04285',
  u'TestData_05025',
  u'TestData_05027',
  u'TestData_05029',
  u'TestData_05041',
  u'TestData_05103',
  u'TestData_05126',
  u'TestData_05324',
  u'TestData_05717'],
 'charliehebdoattack': [u'TestData_00072',
  u'TestData_00077',
  u'TestData_00121',
  u'TestData_00156',
  u'TestData_00172',
  u'TestData_00185',
  u'TestData_00193',
  u'TestData_00249',
  u'TestData_00480',
  u'TestData_01147',
  u'TestData_01162',
  u'TestData_01329',
  u'TestData_02540',
  u'TestData_03260',
  u'TestData_04189'],
 'francetrainatt

In [333]:
# train_test_df.ix['TestData_06110']['topics'] = []

In [350]:
train_test_df.shape

(122, 3)

In [351]:
for topic, idxs in train_test_data.items():
    for idx in idxs:
        train_test_df.ix[idx]['topics'] = [topic]

In [352]:
train_test_df.to_hdf('train_test_df_3.hdf', 'train_test_df')

In [246]:
train_test_df

,bodyText,topics,webPublicationDate
TestData_06126,Details have emerged of a motorcyclist who cha...,[bastilledaytruckattack],15-07-2016
TestData_06155,Eighty-five people are still in hospital follo...,[bastilledaytruckattack],17-07-2016
TestData_06158,The Tunisian delivery driver who killed 84 peo...,[bastilledaytruckattack],17-07-2016
TestData_07514,Police forces across Britain are reviewing the...,[berlinchristmasmarketattack],21-12-2016
TestData_07522,A hunt is under way in Germany for an armed at...,[berlinchristmasmarketattack],21-12-2016
TestData_05027,British people are advised not to travel to Br...,[brusselsattacks],22-03-2016
TestData_05028,Ukip has been criticised by David Cameron and ...,[brusselsattacks],22-03-2016
TestData_05029,Islamic State has claimed responsibility for t...,[brusselsattacks],22-03-2016
TestData_05041,A massive manhunt is under way in Belgium as p...,[brusselsattacks],23-03-2016
TestData_05051,The aftermath of the horrific terrorist attack...,[brusselsattacks],23-03-2016


## Use 7512 for Berlin Christmas market attack / Germany / Paris attacks / Brussels attacks / France / analysis

In [343]:
test_df.iloc[834].bodyText # 19

u'Tunisia is a small country relative to its north African neighbours but the significance of last week\u2019s dreadful terrorist attack on the Bardo museum in central Tunis, which killed 23 people, including one Briton, is huge. This atrocious act reflected all the varied strains and tensions fuelling one of the most momentous struggles of the modern era \u2013 the battle for the heart of Islam. As is frequently noted, Tunisia is the birthplace and home of the Arab Spring, that great, spontaneous uprising against dictatorship and repression that began in late 2010 and spread across the Arab world. In Egypt, Libya, Syria, Yemen and Bahrain, the rule of unelected, abusive regimes was challenged as never before, but only in Tunisia did democracy take lasting root. Elections held after the ignominious collapse of the Ben Ali presidency in 2011 have seen power shared and swapped in exemplary fashion between secular-minded parties and the moderate Islamist movement, Ennahda. That favourite 

In [284]:
# 'tfidf_wvec': [test_tfidf_wvec, unseen_topics2vec.tfidf_wv],
# 'tfidf_fvec': [test_tfidf_fvec, unseen_topics2vec.tfidf_fs],
# 'tfidf_lsi': [test_tfidf_lsi, unseen_topics2vec.tfidf_lsi],

sim = cosine_similarity(
    np.vstack(test_tfidf_wvec),
    np.vstack(unseen_topics2vec.tfidf_wv),
)
N = 5
for i, t in enumerate(unseen_topics2vec.index):
    idx = sim[:, i].argsort()[::-1][:N]
    v = sim[:, i][idx]
    print unseen_topics2vec.index[i], zip(idx[:N], np.round(v[:N], 1))


activism [(2265, 0.5), (4471, 0.5), (152, 0.5), (5445, 0.5), (6796, 0.5)]
bastilledaytruckattack [(6128, 0.80000000000000004), (6098, 0.69999999999999996), (6121, 0.69999999999999996), (6105, 0.69999999999999996), (6126, 0.69999999999999996)]
berlinchristmasmarketattack [(3681, 0.80000000000000004), (7491, 0.80000000000000004), (7521, 0.69999999999999996), (7503, 0.69999999999999996), (7489, 0.69999999999999996)]
brusselsattacks [(5125, 0.80000000000000004), (5019, 0.80000000000000004), (5046, 0.80000000000000004), (3515, 0.80000000000000004), (5102, 0.69999999999999996)]
charliehebdoattack [(71, 0.90000000000000002), (4276, 0.90000000000000002), (60, 0.90000000000000002), (4188, 0.90000000000000002), (120, 0.90000000000000002)]
francetrainattack [(3514, 0.69999999999999996), (1246, 0.69999999999999996), (3198, 0.69999999999999996), (2263, 0.69999999999999996), (6256, 0.69999999999999996)]
munichshooting [(6267, 0.80000000000000004), (6231, 0.80000000000000004), (6236, 0.69999999999999

In [272]:
v

array([ 0.85859394,  0.57481531,  0.56485035,  0.55066684,  0.5251468 ,
        0.45473479,  0.44471953,  0.44246537,  0.42221175,  0.42088665,
        0.4148486 ,  0.41344707,  0.4058871 ,  0.40490635,  0.39717574,
        0.39518468,  0.39454976,  0.3910347 ,  0.38950969,  0.38351464])

In [125]:
test_max_class_df.ix['TestData_04490']

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [262]:
tpvec.index[41]

'ebola'

In [126]:
test_max_class_df.iloc[4489]

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [78]:
test_max_class_df.iloc[4489]

tfidf_fvec    zikavirus
tfidf_wvec    zikavirus
tfidf_lsi         ebola
Name: TestData_04490, dtype: object

In [222]:
sim.argmax(axis=1)

array([151, 134,  29, ...,  72, 136, 136])

In [216]:
Counter(sim.argmax(axis=1))

Counter({0: 14,
         1: 38,
         2: 15,
         3: 12,
         4: 27,
         5: 35,
         6: 5,
         7: 9,
         8: 8,
         9: 32,
         10: 85,
         11: 201,
         12: 6,
         13: 16,
         14: 15,
         15: 2,
         16: 10,
         17: 8,
         18: 12,
         19: 38,
         20: 7,
         21: 16,
         22: 70,
         23: 11,
         24: 20,
         25: 8,
         26: 17,
         27: 44,
         28: 31,
         29: 18,
         30: 5,
         31: 3,
         32: 47,
         33: 30,
         34: 8,
         35: 9,
         36: 32,
         37: 68,
         38: 1,
         39: 3,
         40: 27,
         41: 38,
         42: 11,
         43: 56,
         44: 38,
         45: 19,
         46: 17,
         47: 5,
         48: 69,
         49: 15,
         50: 28,
         51: 2,
         52: 8,
         53: 254,
         54: 17,
         55: 25,
         56: 124,
         57: 26,
         58: 34,
         59: 8,
     

In [221]:
np.where(all_topics2vec.index.str.contains('bastille'))

(array([11]),)

In [259]:
tfidf.idfs[dictionary.token2id['virus']]

7.202270959611463

In [168]:
test_doc_tokens = pd.DataFrame({k: {dictionary.id2token[t]: c for t, c in dictionary.doc2bow(i)} for k, i in zip(test_transformed_text.index, test_transformed_text.str.split())})

In [178]:
test_doc_tokens = test_doc_tokens.T

In [217]:
test_doc_tokens[test_doc_tokens.zika > 0].shape

(10, 84860)

In [ ]:
def score_topic_doc_match(doc_tokens, topic_tokens):
    

In [84]:
unseen_topics_rank

{'activism': [array([6796, 5937,  396, 4906, 4471]),
  array([6796, 6624, 4906,  396, 2265]),
  array([7182, 4421,  170, 3121, 4062])],
 'bastilledaytruckattack': [array([6128,   78, 3188, 6135, 3210]),
  array([6128,   78, 3220, 3188, 3210]),
  array([5142, 3187, 3182, 4340, 7481])],
 'berlinchristmasmarketattack': [array([7491, 6818, 3681, 7489, 6267]),
  array([7491, 6267, 7489, 3681, 6818]),
  array([7489, 3681, 7535, 4253, 7513])],
 'brusselsattacks': [array([3300, 5008, 4458, 5080, 7056]),
  array([3300, 5008, 3554, 5019, 5028]),
  array([2062, 3620, 5028, 5113, 5052])],
 'charliehebdoattack': [array([3124,   45,   67, 4198,   49]),
  array([3124,   45,   49, 4198, 2539]),
  array([3182, 4198, 3873, 2062, 4961])],
 'francetrainattack': [array([3514, 2263, 5028, 5475, 6636]),
  array([3514, 2263, 5028, 5475, 1246]),
  array([6118, 3200, 7210, 2263, 3649])],
 'munichshooting': [array([6268, 6254, 6231, 6267, 6232]),
  array([4031, 6268, 6231, 2129, 6254]),
  array([6239, 6254, 6249

In [319]:
test_df.iloc[3514].bodyText # 19

u'French police have tweeted a picture of the third man they believe carried out the bomb attack on the Stade de France last week, and appealed for help in identifying him. The Police Nationale released the image on Sunday evening, nine days after the bombings, which were part of a series of simultaneous attacks across the French capital that killed 130 people. It followed police raids in which three people suspected of involvement in the attacks were killed. The man\u2019s image was accompanied by a message that read: \u201cThis individual is the third dead perpetrator of one of the attacks carried out on 13 November at the Stade de France.\u201d In Belgium, the prime minister, Charles Michel, said the terror alert would remain at the country\u2019s highest level for a sustained \u201cserious and imminent\u201d threat against the capital, Brussels. Michel said that, like the weekend, authorities fear a Paris-like attack, \u201ceven perhaps at several locations\u201d. He said that cons

In [253]:
test_df[test_df.bodyText.str.contains('Zika')].bodyText.iloc[2]

u'Violent protests at Trump rally in California With the tenor of the general election campaign now clearly defined, Hillary Clinton delivered a stinging rebuke to Donald Trump\u2019s claim to understand international affairs on Thursday. Trump, meanwhile, described Clinton\u2019s clear-headed foreign policy address \u2013 in which she described him as too unstable to be trusted with nuclear launch codes and warned he would take the country down a \u201ctruly dangerous path\u201d \u2013 as \u201cpathetic\u201d and said she should be in jail. Violence later erupted at a Trump rally in California. Reports from San Jose described anti-Trump demonstrators chasing \u2013 and in some cases punching and attacking \u2013 departing Trump supporters, some of whom appeared intent on provoking and fighting. Riot police were deployed to control the crowd. The mayor of San Jose, Sam Liccardo, blamed Trump. \u201cWe don\u2019t appreciate [anyone] utilizing campaign tactics of demagoguery,\u201d he sa

In [245]:
ssim = cosine_similarity(
    np.vstack(test_tfdifwvec)[4489],
    np.vstack(test_tfdifwvec),
)

/home/avsolatorio/ml-ai/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [250]:
ssim.argsort()[0][::-1][:20]

array([4489, 7543, 4760, 4881,  471, 5557,  373,  824, 4456, 4549,  352,
        469,  749, 1906, 1092, 6609, 2255,  950, 2888, 1512])

In [252]:
topics

['activism',
 'afghanistan',
 'aid',
 'algerianhostagecrisis',
 'alqaida',
 'alshabaab',
 'antiwar',
 'arabandmiddleeastprotests',
 'armstrade',
 'australianguncontrol',
 'australiansecurityandcounterterrorism',
 'bastilledaytruckattack',
 'belgium',
 'berlinchristmasmarketattack',
 'bigdata',
 'biometrics',
 'bokoharam',
 'bostonmarathonbombing',
 'britisharmy',
 'brusselsattacks',
 'cameroon',
 'carers',
 'charliehebdoattack',
 'chemicalweapons',
 'clusterbombs',
 'cobra',
 'conflictanddevelopment',
 'controversy',
 'criminaljustice',
 'cybercrime',
 'cyberwar',
 'darknet',
 'dataprotection',
 'debate',
 'defence',
 'deflation',
 'drones',
 'drugs',
 'drugspolicy',
 'drugstrade',
 'earthquakes',
 'ebola',
 'economy',
 'egypt',
 'encryption',
 'energy',
 'espionage',
 'ethics',
 'europeanarrestwarrant',
 'europeancourtofhumanrights',
 'events',
 'extradition',
 'famine',
 'farright',
 'firefighters',
 'forensicscience',
 'france',
 'francetrainattack',
 'freedomofspeech',
 'genevaconv